In [1]:
import sys
sys.path.insert(0, '../')
#from models.model_mrcnn import _default_mrcnn_config, build_default
import torchvision
import torch
from models_pytorch_lightning.generalized_mask_rcnn_pl import LitMaskRCNN
import torchvision.ops.boxes as bops
import torchvision
from torchvision import transforms
import torch
from PIL import Image
import cv2
import numpy as np
import os
import pandas as pd
import os
import pyvips as Vips
from Reinhard import Reinhard
from openslide import OpenSlide
from PIL import Image
from PIL import Image, ImageDraw
from glob import glob
import random

In [2]:
path = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/labels"
label_list = os.listdir(path)

In [3]:
model_name = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/runpod_mrcnn_models/yp2mf3i8_epoch=108-step=872.ckpt"
model = LitMaskRCNN.load_from_checkpoint(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval().to(device)

/home/mahirwar/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/lightning/fabric/utilities/cloud_io.py:55: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/mahirwar/miniconda

LitMaskRCNN(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256,

In [4]:
class_names = ['Cored', 'Diffuse', 'Coarse-Grained', 'CAA']

In [5]:
img_folder =  "/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/images"
img_names =  [ os.path.join(img_folder,label_list[i].replace(".txt",".jpg")) for i in range(len(label_list))]

In [35]:
img_names[0]

'/home/mahirwar/Desktop/Monika/npsad_data/vivek/Datasets/External_Dataset-Others/data/custom/images/cw18-015_0_10_15.jpg'

In [22]:
1536*2/1024

3.0

In [6]:
def zoom_and_crop(image_path, crop_size, zoom_factor):
    """
    Zoom into an image and crop a specific region.
    Parameters:
    - image_path (str): Path to the input image.
    - crop_coordinates (tuple): Coordinates for the crop (x, y, width, height).
    - zoom_factor (float): Factor by which to zoom into the image.
    - output_path (str): Path to save the cropped and zoomed image.
    Returns:
    - None
    """
    # Open the image
    img = Image.open(image_path)
    # Calculate the new dimensions for the zoomed image
    zoomed_width = int(img.width * zoom_factor)
    zoomed_height = int(img.height * zoom_factor)
    # Resize the image (zoom in or out)
    img = img.resize((zoomed_width, zoomed_height), Image.LANCZOS)
    # Crop the zoomed image
    h, w = crop_size
    crops = []
    for i in range(0, zoomed_width, h):
        for j in range(0, zoomed_height, w):
            cropped_img = img.crop((i, j, i + h, j + w))
            crops.append(cropped_img)
    return crops



# Function to initialize normalization based on the reference image
def initialize_normalizer(ref_slide_path):
    print("Initializing normalization with reference slide:", ref_slide_path)
    ref_image = Vips.Image.new_from_file(ref_slide_path)
    normalizer = Reinhard()
    normalizer.fit(ref_image)
    return normalizer


# Function to normalize a single slide
def normalize_slide(target_image, normalizer, filepath, idx, output_dir):
    #print("Normalizing slide:", target_slide_path)
    # Load the target slide
    #target_image = Vips.Image.new_from_file(target_slide_path)
    target_image = Vips.Image.new_from_memory(target_image.data, target_image.shape[1], target_image.shape[0], 3, "uchar")
    # Perform normalization
    normalized_image = normalizer.transform(target_image)
    # Generate the output file path
    #file_name = os.path.basename(target_slide_path).replace(".mrxs", ".tif")
    output_file_path = os.path.join(output_dir, idx+"_" +filepath.split("/")[-1])
    # Save the normalized image
    normalized_image.write_to_file(output_file_path)
    return normalized_image

In [91]:
# Example Usage
image_path = img_names[0]  # Path to the input image
output_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/gladstone_results/"+img_names[0].split("/")[-1] # Path to save the output image
crop_size = (1024, 1024)  # Coordinates for the crop (x, y, width, height)
zoom_factor = 2.0  # Zoom factor (2x zoom)
cropped_img = zoom_and_crop(image_path, crop_size, zoom_factor)

In [126]:
cropped_img[8].save("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/test_img2.jpg")

In [127]:
len(cropped_img)

9

In [7]:
# Path to the reference slide
ref_slide_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png"
# Initialize the normalizer
normalizer = initialize_normalizer(ref_slide_path)

Initializing normalization with reference slide: /gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png


In [38]:
normalize_slide(np.array(cropped_img[2]), normalizer, "/gladstone/finkbeiner/steve/work/data/npsad_data/monika/test_img2_norm.jpg")

NameError: name 'cropped_img' is not defined

In [40]:
def yolo_to_box_coordinates(image_width, image_height, yolo_bboxes):
    """
    Convert YOLO normalized bounding box format to (x_min, y_min, x_max, y_max) format.

    Args:
        image_width (int): Width of the original image.
        image_height (int): Height of the original image.
        yolo_bbox (list or tuple): A list or tuple containing YOLO format [x_center, y_center, width, height].
                                   All values should be normalized (between 0 and 1).

    Returns:
        tuple: Bounding box in (x_min, y_min, x_max, y_max) format.
    """
    transformed_boxes = []
    crop_width, crop_height = 1024, 1024
    yolo_bboxes2 =[]
    for yolo_bbox in yolo_bboxes:
        label, x_center, y_center, width, height = yolo_bbox
        # Convert YOLO normalized coordinates to absolute coordinates
        x_center_abs = x_center * image_width
        y_center_abs = y_center * image_height
        width_abs = width * image_width
        height_abs = height * image_height
        yolo_bboxes2.append([label,x_center_abs,y_center_abs,width_abs,height_abs])
        
    #print(yolo_bboxes2)
    transformed_boxes_all = []
    for i in range(0, image_width, crop_width):
        for j in range(0, image_height, crop_height):
            transformed_boxes = []
            for k in yolo_bboxes2:
                label, x_center_abs, y_center_abs, width_abs, height_abs = k
                if (i <= x_center_abs <= i + crop_width) and (j <= y_center_abs <= j + crop_height):
                    x_center_crop = x_center_abs - i
                    y_center_crop = y_center_abs - j
                    transformed_boxes.append([label, x_center_crop, y_center_crop, width_abs, height_abs])
            transformed_boxes_all.append(transformed_boxes)
    return transformed_boxes_all

In [41]:
label_path = os.path.join(path, label_list[0])
gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
gt_labels = gt_boxes[:,0]

In [42]:
gt_boxes*3072

array([[3072.,  906., 2210.,  148.,  132.],
       [3072., 2996., 2256.,  144.,  168.],
       [3072.,  878., 2822.,  276.,  236.],
       [3072., 1150., 2468.,  212.,  200.]])

In [122]:
transformed_boxes = yolo_to_box_coordinates(1536*2, 1536*2, gt_boxes)

in loop
in loop
in loop
in loop


In [123]:
transformed_boxes

[[],
 [],
 [[1.0, 906.0, 162.0, 148.0, 132.0],
  [1.0, 877.9999999999999, 774.0, 276.0, 236.0]],
 [],
 [],
 [[1.0, 126.0, 420.0, 212.0, 200.0]],
 [],
 [],
 [[1.0, 948.0, 208.0, 144.0, 168.0]]]

In [24]:
len(transformed_boxes)

NameError: name 'transformed_boxes' is not defined

In [137]:
transformed_boxes[8][0]

[1.0, 948.0, 208.0, 144.0, 168.0]

In [143]:
image = Image.open("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/test_img2.jpg")

# Create a drawing object
draw = ImageDraw.Draw(image)
bounding_boxes = transformed_boxes[8][0]

In [144]:
bounding_boxes[1:]

[948.0, 208.0, 144.0, 168.0]

In [145]:
draw.rectangle([bounding_boxes[1:][0],bounding_boxes[1:][1],bounding_boxes[1:][2]+bounding_boxes[1:][0], bounding_boxes[1:][1]+bounding_boxes[1:][3]] ,outline='red')

In [146]:
image.show()


(eog:119638): Gtk-WARNING **: 03:35:15.036: cannot open display: 


In [147]:
image.save("/gladstone/finkbeiner/steve/work/data/npsad_data/monika/test_img2_norm_bbox.jpg")

In [67]:
def prepare_input(image_path):
    image = Image.open(image_path)
    image_float_np = np.float32(image) / 255
    # define the torchvision image transforms
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize(1024, interpolation=Image.LANCZOS),
        torchvision.transforms.ToTensor(),
    ])
    #input_tensor = transform(transforms.ToPILImage()(image))
    input_tensor = transform(image)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_tensor = input_tensor.to(device)
    # Add a batch dimension:
    input_tensor = input_tensor.unsqueeze(0)
    return input_tensor, image_float_np



def get_outputs_nms(input_tensor, model, score_threshold = 0.5, iou_threshold = 0.5):
    with torch.no_grad():
        # forward pass of the image through the modle
        outputs = model(input_tensor)
    boxes = outputs[0]['boxes']
    labels = outputs[0]['labels']
    scores = outputs[0]['scores']
    masks = outputs[0]['masks']
    # Apply score threshold
    keep = scores > score_threshold
    boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
    keep = torchvision.ops.nms(boxes, scores, iou_threshold)
    boxes, labels, scores, masks = boxes[keep], labels[keep], scores[keep], masks[keep]
    scores = list(scores.detach().cpu().numpy())
    masks = list((masks>0.5).detach().cpu().numpy())
    boxes = [[int(i[0]), int(i[1]), int(i[2]), int(i[3])]  for i in boxes.detach().cpu()]
    labels = list(labels.detach().cpu().numpy())
    labels = [class_names[i-1] for i in labels]
    return masks, boxes, labels, scores



def draw_boxes(boxes, labels, classes, image):
    for i, box in enumerate(boxes):
        color = self.colors[labels[i]-1]
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, 2
        )
        cv2.putText(image, classes[i], (int(box[0]), int(box[1] + 30)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 1,
                    lineType=cv2.LINE_AA)
    return image
    
def change_to_numpy_array(input_tensor):
    input_tensor_cpu = input_tensor.squeeze(0).cpu()  # Remove batch dimension and move to CPU if necessary
    # Convert to NumPy array
    numpy_array = input_tensor_cpu.permute(1, 2, 0).numpy()  # Convert to HWC format for visualization
    # Scale back to 0-255 range if needed
    numpy_array = (numpy_array * 255).astype(np.uint8)
    return numpy_array
  
class_names = ['Cored', 'Diffuse', 'Coarse-Grained', 'CAA']
class_to_colors = {'Cored': (255, 0, 0), 'Diffuse' : (0, 0, 255), 'Coarse-Grained': (0,255,0), 'CAA':(225, 255, 0)}
#TODO change this to nas location later
#result_save_dir= "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/gladstone_results/model_results"
colors = np.random.uniform(0, 255, size=(len(class_names), 3))  
    
    
def draw_segmentation_map(image, masks, boxes, labels,scores):
    alpha = 1 
    beta = 0.6 # transparency for the segmentation map
    gamma = 0 # scalar added to each 
    segmentation_map = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    result_masks = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    masks = [ masks[i].squeeze() for i in range(len(masks))]
    for i in range(len(masks)):
        # TODO fix the color segmentation masks
        red_map = np.zeros_like(masks[i]).astype(np.uint8)
        green_map = np.zeros_like(masks[i]).astype(np.uint8)
        blue_map = np.zeros_like(masks[i]).astype(np.uint8)
        
        # apply a randon color mask to each object
        rect_color = (0,0,0)
        color = colors[random.randrange(0, len(colors))]
        red_map[masks[i] == 1], green_map[masks[i] == 1], blue_map[masks[i] == 1]  = class_to_colors[labels[i]]
        result_masks[masks[i] == 1] = 255
        # combine all the masks into a single image
        # change the format of mask to W,H, C

        # segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
        #convert the original PIL image into NumPy format
        image = np.array(image)
        # convert from RGB to OpenCV BGR format
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # apply mask on the image
        # cv2.addWeighted(image, alpha, segmentation_map, beta, gamma, image)
        # draw the bounding boxes around the objects
        cv2.rectangle(image, boxes[i][0], boxes[i][1], color=rect_color, 
                    thickness=2)
        # Get the centre coords of the rectangle-plaque-detection/src/visualizat
        x1 = boxes[i][0][0]
        y1 = boxes[i][0][1]
        x2 = boxes[i][1][0]
        y2 = boxes[i][1][1]
        x = int((x1 + x2) / 2)
        y = int((y1+y2) / 2)

        
        # put the label text above the objects
        cv2.putText(image , labels[i]+str(np.round(scores[i],2)), (x1, y1-20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 
                    thickness=2, lineType=cv2.LINE_AA)
        
        # Convert Back
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image, result_masks



def bbox_iou(box1, box2):
    """
    Returns the IoU of two bounding boxes
    """
    # Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1
    b2_x1, b2_y1, b2_x2, b2_y2 = box2

    # get the corrdinates of the intersection rectangle
    inter_rect_x1 = np.max(b1_x1, b2_x1)
    inter_rect_y1 = np.max(b1_y1, b2_y1)
    inter_rect_x2 = np.min(b1_x2, b2_x2)
    inter_rect_y2 = np.min(b1_y2, b2_y2)
    # Intersection area
    inter_area = np.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * np.clamp(
        inter_rect_y2 - inter_rect_y1 + 1, min=0
    )
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)
    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)
    return iou


In [68]:
crop_size = (1024, 1024)  # Coordinates for the crop (x, y, width, height)
zoom_factor = 2.0  # Zoom factor (2x zoom)
for idx, img_name in enumerate(img_names):
    #print(img_name)
    #img_name = img_names[0]
    cropped_imgs = zoom_and_crop(img_name, crop_size, zoom_factor, output_path)
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    transformed_boxes = yolo_to_box_coordinates(1536*2, 1536*2, gt_boxes)
    cnt = 0
    for cropped_img, transformed_box in zip(cropped_imgs,transformed_boxes):
        cnt = cnt+1
        if len(transformed_box)>0:
            normalize_slide(np.array(cropped_img), normalizer,img_name,str(cnt), "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images")
            # File path to save the bounding box coordinates
            bbox_output_file ="/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/bboxes/"+str(cnt)+"_"+img_name.split("/")[-1].replace("jpg","txt")
            # Save bounding box coordinates to the file
            with open(bbox_output_file, "w") as f:
                for box in transformed_box:
                    # Convert each bounding box to a space-separated string
                    line = " ".join(map(str, box)) + "\n"
                    f.write(line)

NameError: name 'output_path' is not defined

In [96]:
crop_size = (1024, 1024)  # Coordinates for the crop (x, y, width, height)
zoom_factor = 2.0  # Zoom factor (2x zoom)
for idx, img_name in enumerate(img_names):
    cropped_imgs = zoom_and_crop(img_name, crop_size, zoom_factor)
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    transformed_boxes = yolo_to_box_coordinates(1536*2, 1536*2, gt_boxes)
    cnt = 0
    for cropped_img, transformed_box in zip(cropped_imgs,transformed_boxes):
        cnt = cnt+1
        if len(transformed_box)>0:
            #image = Image.open("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images/"+str(cnt)+"_"+img_name.split("/")[-1])          
            # Create a drawing object
            draw = ImageDraw.Draw(cropped_img)
            for box in transformed_box:
                bounding_boxes = box
                draw.rectangle([bounding_boxes[1:][0],bounding_boxes[1:][1],bounding_boxes[1:][2]+bounding_boxes[1:][0], bounding_boxes[1:][1]+bounding_boxes[1:][3]] ,outline='red')
            cropped_img.save("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/image_bbox/"+str(cnt)+"_"+img_name.split("/")[-1])

KeyboardInterrupt: 

In [ ]:
for idx, img_name in enumerate(img_names):
    cropped_imgs = zoom_and_crop(img_name, crop_size, zoom_factor, output_path)
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    transformed_boxes = yolo_to_box_coordinates(1536*2, 1536*2, gt_boxes)
    cnt = 0
    for cropped_img, transformed_box in zip(cropped_imgs,transformed_boxes):
        cnt = cnt+1
        if len(transformed_box)>0:
            cropped_img.save("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/unnormalized_images/"+str(cnt)+"_"+img_name.split("/")[-1])

In [160]:
normalized_image = normalize_slide(np.array(cropped_img), normalizer,img_name,str(cnt), "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data")

In [161]:
normalized_image

In [69]:
#imgs = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/unnormalized_images","*"))
imgs = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images","*"))
lbls = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/bboxes","*"))


In [70]:
imgs = sorted(imgs)
lbls = sorted(lbls)

In [11]:
img_names_list = []
pred_label_list = []
act_label_list = []
iou_list = []

for img_path, lbl_path in zip(imgs,lbls):
    #img_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images/1_cw17-020_0_9_17.jpg"
    #lbl_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images/1_cw17-020_0_9_17.txt"
    #print(img_path, lbl_path)
    input_tensor, image_float_np = prepare_input(img_path)
    masks, boxes, labels, scores = get_outputs_nms(input_tensor, model, 0.4, 0.25)
    
    gt_boxes = np.loadtxt(lbl_path).reshape(-1, 5)
    for i in range(len(gt_boxes)):
        img_names_list.append(img_path.split("/")[-1])
        act_label_list.append(gt_boxes[i][0])
        pred_exist = -1
        for j in range(len(boxes)):
            #iou_val = bbox_iou(gt_boxes_coords[i], boxes[j])
            #print(iou_val)
            box1 = torch.tensor([gt_boxes[i][1:]], dtype=torch.float)
            box2 = torch.tensor([boxes[j]], dtype=torch.float)
            iou = bops.box_iou(box1, box2)
            if iou>0:
                pred_exist = labels[j]
                continue
        #if pred_exist==None:
        pred_label_list.append(pred_exist)

NameError: name 'c' is not defined

In [16]:
gt_boxes

array([[  0., 297., 274., 594., 548.]])

In [71]:
result_save_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/model_output_gt"
for img_path, lbl_path in zip(imgs,lbls):
    input_tensor, image_float_np = prepare_input(img_path)
    masks, boxes, labels, scores = get_outputs_nms(input_tensor, model, 0.5, 0.25)
    gt_boxes = np.loadtxt(lbl_path).reshape(-1, 5)
    image_1024 = change_to_numpy_array(input_tensor)
    #result_img, result_masks = draw_segmentation_map(image_1024, masks, boxes, labels, scores)
    detection_img_name = img_path.split("/")[-1] + "_detection.png"
    detection_save_path = os.path.join(result_save_dir, detection_img_name)
    print(detection_save_path)
    #bgr_img = cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR)
    #cv2.imwrite(detection_save_path, bgr_img)
    #bgr_img = Image.open(detection_save_path)
    bgr_img = Image.fromarray(image_1024)
    draw = ImageDraw.Draw(bgr_img)
    for box in boxes:
        bounding_boxes = box
        draw.rectangle([bounding_boxes[0],bounding_boxes[1],bounding_boxes[2], bounding_boxes[3]] ,outline='red')

    for gt_box in gt_boxes:
        bounding_boxes = gt_box
        draw.rectangle([bounding_boxes[1:][0],bounding_boxes[1:][1],bounding_boxes[1:][2]+bounding_boxes[1:][0], bounding_boxes[1:][1]+bounding_boxes[1:][3]] ,outline='green')
    bgr_img.save(detection_save_path)
        

/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/model_output_gt/1_48283-3_UTSW_temporal_6E10_0_0_15.jpg_detection.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/model_output_gt/1_48283-3_UTSW_temporal_6E10_0_11_15.jpg_detection.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/model_output_gt/1_48283-3_UTSW_temporal_6E10_0_11_19.jpg_detection.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/model_output_gt/1_48283-3_UTSW_temporal_6E10_0_11_6.jpg_detection.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/model_output_gt/1_48283-3_UTSW_temporal_6E10_0_12_23.jpg_detection.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/model_output_gt/1_48283

In [59]:
boxes


[[395, 352, 493, 456],
 [657, 82, 712, 139],
 [20, 936, 145, 1024],
 [0, 438, 108, 592]]

In [52]:
print("Image type:", type(result_img))
print("Image shape:", result_img.shape)
print("Image dtype:", result_img.dtype)


Image type: <class 'numpy.ndarray'>
Image shape: (1024, 1024, 3)
Image dtype: uint8


In [28]:
boxes

[]

In [26]:
(gt_box[1].astype(int), gt_box[2].astype(int))

(297, 274)

In [27]:
 (gt_box[1].astype(int)+gt_box[3].astype(int), gt_box[2].astype(int)+gt_box[4].astype(int))

(891, 822)

In [247]:
df1 = pd.DataFrame({"img_name":img_names_list, "pred_label":pred_label_list, "act_label":act_label_list})
#df["pred_label1"] = np.where(df["pred_label"].isin(["Coarse-Grained","Cored","Diffuse"]),"Cored",df["pred_label"])


In [22]:
colors[0]

array([ 27.79426091, 215.09401275,  39.1353531 ])

In [250]:
len(df[df["pred_label"]!=-1])

40

In [240]:
df["img_name"].nunique()

1391

In [227]:
df.iloc[0]["pred_label"]

In [214]:
df["act_label"]= df["act_label"].apply(lambda  l : if l==0 return "CAA" else return "Cored")

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0

In [62]:
consensus_pth=  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/strict_agreed_by_2.csv"
consensus_data = pd.read_csv(consensus_pth)

In [65]:
consensus_data["imagename"]

0        48283-3_UTSW_temporal_6E10/48283-3_UTSW_tempor...
1        48283-3_UTSW_temporal_6E10/48283-3_UTSW_tempor...
2        48283-3_UTSW_temporal_6E10/48283-3_UTSW_tempor...
3        48283-3_UTSW_temporal_6E10/48283-3_UTSW_tempor...
4        48283-3_UTSW_temporal_6E10/48283-3_UTSW_tempor...
                               ...                        
20094                        cw18-015/cw18-015_9_32_36.jpg
20095                        cw18-015/cw18-015_9_32_37.jpg
20096                        cw18-015/cw18-015_9_32_38.jpg
20097                        cw18-015/cw18-015_9_32_39.jpg
20098                        cw18-015/cw18-015_9_32_41.jpg
Name: imagename, Length: 20099, dtype: object

In [14]:
image_details_csv_path =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/image_details_phase1.csv"
image_details_csv= pd.read_csv(image_details_csv_path)

In [25]:
image_details_csv

,imagename,source,tile_column,tile_row,image coordinates (xywh),blob coordinates (xywh),blob size
0,NA4963UCDtemporalgyri _4G8_13_23_0.jpg,NA4963UCDtemporalgyri _4G8,23,13,[ 0 0 256 256],[ 0 0 54 42],1812
1,NA4963UCDtemporalgyri _4G8_13_23_1.jpg,NA4963UCDtemporalgyri _4G8,23,13,[697 0 256 256],[752 0 146 94],9528
2,NA4963UCDtemporalgyri _4G8_13_23_2.jpg,NA4963UCDtemporalgyri _4G8,23,13,[449 0 256 256],[518 54 118 124],9684
3,NA4963UCDtemporalgyri _4G8_13_23_3.jpg,NA4963UCDtemporalgyri _4G8,23,13,[1280 0 256 256],[1492 60 34 50],1172
4,NA4963UCDtemporalgyri _4G8_13_23_4.jpg,NA4963UCDtemporalgyri _4G8,23,13,[331 0 256 256],[410 68 98 110],5696
...,...,...,...,...,...,...,...
526526,cw17-020_29_30_12.jpg,cw17-020,30,29,[1252 1171 256 256],[1228 1274 304 50],5468
526527,cw17-020_29_30_13.jpg,cw17-020,30,29,[ 152 1177 256 256],[ 2 1278 556 54],15364
526528,cw17-020_29_30_14.jpg,cw17-020,30,29,[ 695 1174 256 256],[ 248 1274 1150 56],5472
526529,cw17-020_29_30_15.jpg,cw17-020,30,29,[ 113 1166 256 256],[ 160 1282 162 24],12340


In [21]:
strict_agreed_by_2_csv_path =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/strict_agreed_by_2.csv"
strict_agreed_by_2= pd.read_csv(strict_agreed_by_2_csv_path)

In [26]:
strict_agreed_by_2["source"] = strict_agreed_by_2["imagename"].apply(lambda l : l.split("/")[0])
strict_agreed_by_2["imagename"] = strict_agreed_by_2["imagename"].apply(lambda l : l.split("/")[1])

In [29]:
image_details_agreed =  pd.merge(strict_agreed_by_2,image_details_csv, on = "imagename", how = "left")

In [32]:
img_all = image_details_csv["imagename"].unique()

In [33]:
len(img_all)

526531

In [35]:
img_all = image_details_agreed["imagename"].unique()

In [36]:
len(img_all)

20099

In [44]:
img_all

array(['48283-3_UTSW_temporal_6E10_0_15_1.jpg',
       '48283-3_UTSW_temporal_6E10_0_15_10.jpg',
       '48283-3_UTSW_temporal_6E10_0_15_11.jpg', ...,
       'cw18-015_9_32_38.jpg', 'cw18-015_9_32_39.jpg',
       'cw18-015_9_32_41.jpg'], dtype=object)

In [47]:
l = [x.split("/")[-1] for x in img_names]
l = [x for x in l if x in img_all]

In [48]:
len(l)

1

In [49]:
len(img_names)

659

In [50]:
len(l)

1

In [87]:
img_names_selected = [os.path.join(img_folder,x) for x in l]

In [93]:
crop_size = (1024, 1024)  # Coordinates for the crop (x, y, width, height)
zoom_factor = 2.0  # Zoom factor (2x zoom)
for idx, img_name in enumerate(img_names_selected):
    #print(img_name)
    #img_name = img_names[0]
    cropped_imgs = zoom_and_crop(img_name, crop_size, zoom_factor)
    label_path = os.path.join(path, label_list[idx])
    gt_boxes = np.loadtxt(label_path).reshape(-1, 5)
    transformed_boxes = yolo_to_box_coordinates(1536*2, 1536*2, gt_boxes)
    cnt = 0
    for cropped_img, transformed_box in zip(cropped_imgs,transformed_boxes):
        cnt = cnt+1
        if len(transformed_box)>0:
            normalize_slide(np.array(cropped_img), normalizer,img_name,str(cnt), "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/normalized_selected_images")
            # File path to save the bounding box coordinates
            bbox_output_file ="/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/selected_bboxes/"+str(cnt)+"_"+img_name.split("/")[-1].replace("jpg","txt")
            # Save bounding box coordinates to the file
            with open(bbox_output_file, "w") as f:
                for box in transformed_box:
                    # Convert each bounding box to a space-separated string
                    line = " ".join(map(str, box)) + "\n"
                    f.write(line)

In [94]:
imgs_sel = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/normalized_selected_images","*"))
lbls_sel = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/selected_bboxes","*"))

In [110]:
img_names_list = []
pred_label_list = []
act_label_list = []
iou_list = []
pred_labels_all= []
gt_labels = []
pred_labels = []

for img_path, lbl_path in zip(imgs,lbls):
    #img_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images/1_cw17-020_0_9_17.jpg"
    #lbl_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/normalized_data/images/1_cw17-020_0_9_17.txt"
    #print(img_path, lbl_path)
    input_tensor, image_float_np = prepare_input(img_path)
    masks, boxes, labels, scores = get_outputs_nms(input_tensor, model, 0.4, 0.25)
    
    gt_boxes = np.loadtxt(lbl_path).reshape(-1, 5)
    img_names_list.append(img_path.split("/")[-1])
    act_label_list.append(gt_boxes[:,0])
    pred_label_list.append(labels)
    """
    for i in range(len(gt_boxes)):
        #img_names_list.append(img_path.split("/")[-1])
        #act_label_list.append(gt_boxes[i][0])
        #pred_exist = -1
        for j in range(len(boxes)):
            #iou_val = bbox_iou(gt_boxes_coords[i], boxes[j])
            #print(iou_val)
            box1 = torch.tensor([gt_boxes[i][1:]], dtype=torch.float)
            box2 = torch.tensor([boxes[j]], dtype=torch.float)
            iou = bops.box_iou(box1, box2)
            #if iou>0:
                #print(iou)
            #    pred_exist = labels[j]
            #    continue
        #pred_label_list.append(pred_exist)
    """

In [112]:
act_label_list

[array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([1.]),
 array([0., 1.]),
 array([1.]),
 array([1.]),
 array([0., 1., 1.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1., 1.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([1.]),
 array([0.]),
 array([1.]),
 array([1.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0., 0.]),
 array([0., 0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([0.]),
 array([1.]),
 array([0.]),
 array([1., 0.]),
 array([1.]),
 array([0., 1.]),
 array([0.]),
 array([0., 0., 0., 0.]),
 array([0.]),
 array([1.]),
 array([1.]),
 arr

In [113]:
pred_label_list

[[],
 ['Diffuse', 'Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse'],
 [],
 ['Diffuse', 'Diffuse', 'Diffuse', 'Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Diffuse'],
 ['Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse'],
 ['Coarse-Grained'],
 ['Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Diffuse'],
 ['Coarse-Grained', 'Diffuse'],
 [],
 ['Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Diffuse', 'Coarse-Grained'],
 ['Diffuse', 'Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Coarse-Grained'],
 ['Diffuse', 'Diffuse'],
 ['Diffuse'],
 ['Diffuse'],
 ['Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse',
  'Diffuse'],
 ['Diffuse', 'Diffuse', 'Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Diffuse', 'Diffuse'],
 ['Coarse-Grained', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Diffuse', 'Diffuse'],
 ['Diffuse', 'Diffuse', 'Coarse-Grai

In [111]:
len(act_label_list)

1391

In [102]:
len([x for x in pred_label_list if x!=-1])

78

In [104]:
pred_label_list

[-1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Coarse-Grained',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 'Diffuse',
 'Diffuse',
 -1,
 'Diffuse',
 -1,
 -1,
 'Diffuse',
 -1,
 -1,
 -1,
 -1,

In [10]:
strict_agreed_by_2_csv_path =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/strict_agreed_by_2.csv"
strict_agreed_by_2= pd.read_csv(strict_agreed_by_2_csv_path)

In [20]:
strict_agreed_by_2["imagename"].unique()

array(['48283-3_UTSW_temporal_6E10/48283-3_UTSW_temporal_6E10_0_15_1.jpg',
       '48283-3_UTSW_temporal_6E10/48283-3_UTSW_temporal_6E10_0_15_10.jpg',
       '48283-3_UTSW_temporal_6E10/48283-3_UTSW_temporal_6E10_0_15_11.jpg',
       ..., 'cw18-015/cw18-015_9_32_38.jpg',
       'cw18-015/cw18-015_9_32_39.jpg', 'cw18-015/cw18-015_9_32_41.jpg'],
      dtype=object)

In [ ]:
strict_agreed_by_2[""]

In [13]:
image_details_csv

NameError: name 'image_details_csv' is not defined

## Test agreed data

In [58]:
image_details_csv

,imagename,source,tile_column,tile_row,image coordinates (xywh),blob coordinates (xywh),blob size
0,NA4963UCDtemporalgyri _4G8_13_23_0.jpg,NA4963UCDtemporalgyri _4G8,23,13,[ 0 0 256 256],[ 0 0 54 42],1812
1,NA4963UCDtemporalgyri _4G8_13_23_1.jpg,NA4963UCDtemporalgyri _4G8,23,13,[697 0 256 256],[752 0 146 94],9528
2,NA4963UCDtemporalgyri _4G8_13_23_2.jpg,NA4963UCDtemporalgyri _4G8,23,13,[449 0 256 256],[518 54 118 124],9684
3,NA4963UCDtemporalgyri _4G8_13_23_3.jpg,NA4963UCDtemporalgyri _4G8,23,13,[1280 0 256 256],[1492 60 34 50],1172
4,NA4963UCDtemporalgyri _4G8_13_23_4.jpg,NA4963UCDtemporalgyri _4G8,23,13,[331 0 256 256],[410 68 98 110],5696
...,...,...,...,...,...,...,...
526526,cw17-020_29_30_12.jpg,cw17-020,30,29,[1252 1171 256 256],[1228 1274 304 50],5468
526527,cw17-020_29_30_13.jpg,cw17-020,30,29,[ 152 1177 256 256],[ 2 1278 556 54],15364
526528,cw17-020_29_30_14.jpg,cw17-020,30,29,[ 695 1174 256 256],[ 248 1274 1150 56],5472
526529,cw17-020_29_30_15.jpg,cw17-020,30,29,[ 113 1166 256 256],[ 160 1282 162 24],12340


In [59]:
l = [x.split("/")[-1] for x in img_names]

In [62]:
image_details_csv["image_present"] = image_details_csv["imagename"].apply(lambda x : 1 if x in l else 0)

In [63]:
image_details_csv[image_details_csv["image_present"]==1]

,imagename,source,tile_column,tile_row,image coordinates (xywh),blob coordinates (xywh),blob size,image_present
20909,NA4998UCDtemporalgyri _4G8_0_15_3.jpg,NA4998UCDtemporalgyri _4G8,15,0,[ 21 701 256 256],[128 806 42 46],1044,1
52735,NA4003UCDtemporalgyri _4G8_0_23_9.jpg,NA4003UCDtemporalgyri _4G8,23,0,[441 885 256 256],[514 962 110 102],932,1
52796,NA4003UCDtemporalgyri _4G8_0_25_20.jpg,NA4003UCDtemporalgyri _4G8,25,0,[619 752 256 256],[726 860 42 40],3704,1
52983,NA4003UCDtemporalgyri _4G8_0_26_10.jpg,NA4003UCDtemporalgyri _4G8,26,0,[847 49 256 256],[946 148 58 58],1688,1
52990,NA4003UCDtemporalgyri _4G8_0_26_17.jpg,NA4003UCDtemporalgyri _4G8,26,0,[234 230 256 256],[336 342 52 32],1188,1
...,...,...,...,...,...,...,...,...
501371,cw17-020_0_11_17.jpg,cw17-020,11,0,[690 937 256 256],[ 784 1036 68 58],1128,1
501428,cw17-020_0_9_17.jpg,cw17-020,9,0,[1167 1052 256 256],[1236 1144 118 72],900,1
501476,cw17-020_0_20_28.jpg,cw17-020,20,0,[1173 385 256 256],[1274 488 54 50],2372,1
501782,cw17-020_0_22_27.jpg,cw17-020,22,0,[316 275 256 256],[404 372 80 62],2692,1


In [52]:
def zoom_and_crop(image_path, crop_coords, zoom_factor, blob_coords):
    """
    Zoom into an image and crop a specific region.
    Parameters:
    - image_path (str): Path to the input image.
    - crop_coordinates (tuple): Coordinates for the crop (x, y, width, height).
    - zoom_factor (float): Factor by which to zoom into the image.
    - output_path (str): Path to save the cropped and zoomed image.
    Returns:
    - None
    """
    # Open the image
    img = Image.open(image_path)
    # Calculate the new dimensions for the zoomed image
    zoomed_width = int(img.width * zoom_factor)
    zoomed_height = int(img.height * zoom_factor)
    # Resize the image (zoom in or out)
    img = img.resize((zoomed_width, zoomed_height), Image.LANCZOS)
    # Crop the zoomed image
    # Adjust crop coordinates to match the zoomed image
    x, y, width, height = crop_coords
    zoomed_x = int(x * zoom_factor)
    zoomed_y = int(y * zoom_factor)
    zoomed_width = int(width * zoom_factor)
    zoomed_height = int(height * zoom_factor)
    
    cropped_img = img.crop((zoomed_x, zoomed_y, zoomed_x + zoomed_width, zoomed_y + zoomed_height))
    
    blob_x, blob_y, blob_w, blob_h = blob_coords
    
    b_zoomed_x = int( (blob_x * zoom_factor)-zoomed_x)
    b_zoomed_y = int( (blob_y * zoom_factor)-zoomed_y)
    b_zoomed_width = int(blob_w * zoom_factor)
    b_zoomed_height = int(blob_h * zoom_factor)
    #zoomed_blob_coords = [b_zoomed_x,b_zoomed_y,b_zoomed_width,b_zoomed_height]
    scale_x = 1024 / 512
    scale_y = 1024 / 512
    zoomed_blob_coords = [b_zoomed_x*scale_x,   b_zoomed_y*scale_y,  b_zoomed_x*scale_x + b_zoomed_width*scale_x,  b_zoomed_y*scale_y + b_zoomed_height*scale_y]
    return cropped_img, zoomed_blob_coords

In [53]:
def change_to_numpy_array(input_tensor):
    input_tensor_cpu = input_tensor.squeeze(0).cpu()  # Remove batch dimension and move to CPU if necessary
    # Convert to NumPy array
    numpy_array = input_tensor_cpu.permute(1, 2, 0).numpy()  # Convert to HWC format for visualization
    # Scale back to 0-255 range if needed
    numpy_array = (numpy_array * 255).astype(np.uint8)
    return numpy_array

In [54]:
# Function to initialize normalization based on the reference image
def initialize_normalizer(ref_slide_path):
    print("Initializing normalization with reference slide:", ref_slide_path)
    ref_image = Vips.Image.new_from_file(ref_slide_path)
    normalizer = Reinhard()
    normalizer.fit(ref_image)
    return normalizer


# Function to normalize a single slide
def normalize_slide(target_image, normalizer):
    #print("Normalizing slide:", target_slide_path)
    # Load the target slide
    #target_image = Vips.Image.new_from_file(target_slide_path)
    target_image = Vips.Image.new_from_memory(target_image.data, target_image.shape[1], target_image.shape[0], 3, "uchar")
    # Perform normalization
    normalized_image = normalizer.transform(target_image)
    # Generate the output file path
    #file_name = os.path.basename(target_slide_path).replace(".mrxs", ".tif")
    #output_file_path = os.path.join(output_dir, idx+"_" +filepath.split("/")[-1])
    # Save the normalized image
    #normalized_image.write_to_file(output_file_path)
    return normalized_image

In [55]:
# Path to the reference slide
ref_slide_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png"
# Initialize the normalizer
normalizer = initialize_normalizer(ref_slide_path)

Initializing normalization with reference slide: /gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi_v2/XE07-047_1_AmyB_1/image/XE07-047_1_AmyB_1_8381x_120830y_image.png


In [ ]:
image_dataset =  "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/External_Dataset-Others/data/CERAD/1536_tiles"
zoom_factor = 2
def run_prediction(row):
    print(row["id1"], "... of 61424")
    image_path = os.path.join(image_dataset, row["source"], "0",str(row["tile_row"]),str(row["tile_column"])+".jpg")
    crop_coords = row["image coordinates (xywh)"].replace("[","").replace("]","").split()
    crop_coords = [int(x) for x in crop_coords]
    blob_coords = row["blob coordinates (xywh)"].replace("[","").replace("]","").split()
    blob_coords = [int(x) for x in blob_coords]
    
    #print(crop_coords)
    #print(blob_coords)
    cored_gt = row["cored"]
    diffuse_gt = row["diffuse"]
    CAA_gt =  row["CAA"]
    
    cropped_img, zoomed_blob_coords = zoom_and_crop(image_path, crop_coords, zoom_factor,blob_coords)
    #print(cropped_img.shape)
    norm_img = normalize_slide(np.array(cropped_img), normalizer)
    input_tensor, image_float_np = prepare_input(np.array(norm_img))
    masks, boxes, labels, scores = get_outputs_nms(input_tensor, model, 0.6, 0.25)
    cored_pred = 0
    cg_pred = 0
    diffuse_pred = 0
    caa_pred = 0
    label_list= []
    
    #zoomed_blob_coords = [260, 88,260+ 92,88+ 68]
    #print(zoomed_blob_coords)
    for j in range(len(boxes)):
        box1 = torch.tensor([zoomed_blob_coords], dtype=torch.float)
        box2 = torch.tensor([boxes[j]], dtype=torch.float)
        iou = bops.box_iou(box1, box2)
        #print(iou)
        if iou>0:
            label_list.append(labels[j])
    return {"all_labels":labels, "matched_labels" :label_list,  "scores":scores}